# Imports

In [1]:
import argparse

import os
import random
import time
import numpy as np
import pandas as pd

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor

from joblib import dump, load

## Command-line Arguments Definitions

In [2]:
parser = argparse.ArgumentParser()
 

parser.add_argument('--data_path', type=str, default='/home/gean/nns_performance_prediction/meta_datasets/', 
                    help='location of the dataset')    
parser.add_argument('--model_path', type=str, default='/home/gean/nns_performance_prediction/saved_models/', 
                    help='path to save the trained models')
parser.add_argument('--results_path', type=str, default='/home/gean/nns_performance_prediction/results/fast/test12/', 
                    help='location of the results directory')    
parser.add_argument('--target', type=str, default='acc_valid', 
                    help='target of the training/test')
#default = 100
parser.add_argument('--n_iter_rs', type=int, default=2,
                    help='number of iterations for random search')
parser.add_argument('--cv_inner', type=int, default=3,
                    help='number of partitions for the inner split of nested cross-validation')
#config fast/test12
parser.add_argument('--scoring_rs', type=str, default='r2', 
                    help='[neg_mean_absolute_error, neg_mean_squared_error, r2, None]')

#'+' == 1 or more, '*' == 0 or more, '?' == 0 or 1.
# parser.add_argument('--dataset', type=str, default=['cifar10valid', 'cifar100', 'imagenet16_120'], nargs='+', 
parser.add_argument('--dataset', type=str, default=['imagenet16_120'], nargs='+', 
                    help='one of the datasets from nasbench201, being cifar10valid, cifar100, or imagenet16_120')
#minimal efford
parser.add_argument('--data_subset', type=int, default=[4, 108, 200], nargs='+', 
# parser.add_argument('--data_subset', type=int, default=[108, 200], nargs='+', 
                    help='one of the subsets from nasbench201 with 1, 4, 12, 36, 108, or 200 epochs')
parser.add_argument('--seed', type=int, default=[0, 1, 10, 42, 100, 123, 666, 1000, 1234, 12345], nargs='+', 
# parser.add_argument('--seed', type=int, default=[0, 1], nargs='+', 
                    help='seeds used for all the random procedures') 
parser.add_argument('--train_size', type=int, default=[43, 86, 129, 172, 344, 860], nargs='+', 
# parser.add_argument('--train_size', type=int, default=[43, 86], nargs='+', 
                    help='[Int, Int...] representing the total number of train samples')
parser.add_argument('--estimators', type=str, default=['linear_regression', 'sgd', 'lasso', 'bayesian_ridge', 'knn', 'dt', 'svm', 
                                                       'mlp', 'random_forest', 'ada_boost', 'gradient_boost', 'dummy'], nargs='+', 
                    help='list of sklearn estimators to be used for training') 
#remaining 'id_arch', 'conv_num_layers', 'conv_kernel_min', 'conv_kernel_max', 'conv_kernel_mode', 'avg_pool_num_layers'
# 'params', 'acc_valid': 'acc_test'
parser.add_argument('--features_drop', type=str, default=['str_arch', 'layers_all', 'adjacency_matrix', 'dataset', 'epoch', 
                                                          'skip_connection_num_layers', 'zeroize_num_layers', 'flops', 'latency', 
                                                          'time_train', 'time_valid', 'time_test', 'loss_train', 'loss_valid', 
                                                          'loss_test', 'acc_train'], nargs='+', 
                    help='list of features to drop from nasbench201')

args, unknown = parser.parse_known_args() 

In [101]:
def set_default_seed(seed, verbose=True):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
    if verbose: print("\n###### set_default_seed() ######\n")

In [102]:
def get_preprocessed_numpy_dataset(dataset, data_subset, verbose=True):
    df_whole = pd.read_csv(str(args.data_path + 'nasbench201_' + str(dataset) + '_' + str(data_subset) + 'epochs.csv'), index_col=0)
    df_whole = df_whole.reset_index(drop=True)
    df_whole.drop(args.features_drop, axis=1, inplace=True)

    df_y = df_whole[args.target]
    df_X = df_whole.drop([args.target], axis = 1)
    X = df_X.to_numpy()
    y = df_y.to_numpy()
    
    if verbose:
        print("###### get_preprocessed_numpy_dataset() ######")
        print("df_whole after pre-processing:", df_whole.head())
        print("df_X:", df_X.head(2))
        print("df_y:", df_y.head(2))
        print("X numpy:", X[:2])
        print("X numpy shape:", X.shape)
        print("y numpy:", y[:2])
        print("y numpy shape:", y.shape, "\n")
        
    return X, y

In [103]:
def get_estimators(seed, verbose=True):
    estimators_case_insensitive = [el.lower() for el in args.estimators]
    estimators = []
    
    if ("linear_regression" in estimators_case_insensitive):
        estimators.append(LinearRegression(n_jobs=-1))
    
    if ("sgd" in estimators_case_insensitive):
        estimators.append(SGDRegressor(random_state=seed))
        
    if ("lasso" in estimators_case_insensitive):
        estimators.append(Lasso(random_state=seed))
    
    if ("bayesian_ridge" in estimators_case_insensitive):
        estimators.append(BayesianRidge())

    if ("knn" in estimators_case_insensitive):
        estimators.append(KNeighborsRegressor(n_jobs=-1))
    
    if ("dt" in estimators_case_insensitive):
        estimators.append(DecisionTreeRegressor(random_state=seed))
        
    if ("svm" in estimators_case_insensitive):
        estimators.append(SVR())
        
    if ("mlp" in estimators_case_insensitive):
        estimators.append(MLPRegressor(random_state=seed))
        
    if ("random_forest" in estimators_case_insensitive):
        estimators.append(RandomForestRegressor(n_jobs=-1, random_state=seed))
        
    if ("ada_boost" in estimators_case_insensitive):
        estimators.append(AdaBoostRegressor(random_state=seed))
        
    if ("gradient_boost" in estimators_case_insensitive):
        estimators.append(GradientBoostingRegressor(random_state=seed))
        
    if ("dummy" in estimators_case_insensitive):
        estimators.append(DummyRegressor())
    
    if verbose: 
        print("\n###### get_estimators() ######")
        print(estimators, "\n")
    
    return estimators

In [104]:
def get_estimators_hyperparameters_to_search(verbose=True):
    #linear regression
    hp_lr = {'fit_intercept': [False, True],
        'normalize': [False, True]}

    #stochastic gradient descent
    hp_sgd = {'loss': ["squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"],
         'penalty': ["l2", "l1", "elasticnet"],
         'fit_intercept': [False, True],
         'max_iter': [1000, 3000, 9000],
         'shuffle': [False, True],
         'learning_rate': ["constant", "optimal", "invscaling", "adaptive"],
         'early_stopping': [False, True],
         'n_iter_no_change': [5, 15, 45],
         'warm_start': [False, True]}
    
    #lasso
    hp_lasso = {'fit_intercept': [False, True],
            'normalize': [False, True],
           'max_iter': [1000, 3000, 9000],
           'warm_start': [False, True],
           'positive': [False, True],
           'selection': ["cyclic", "random"],
            'tol': [0.001, 0.0001, 0.00001]}
    
    #bayesian ridge
    hp_bayesian = {'n_iter': [1000, 3000, 9000],
              'tol': [0.001, 0.0001, 0.00001],
              'compute_score': [False, True],
              'fit_intercept': [False, True],
                  'normalize': [False, True]}

    #k-nearest neighbors
    hp_knn = {'n_neighbors': list(range(1, 26)),
              'weights': ["uniform", "distance"],
         'algorithm': ["auto", "ball_tree", "kd_tree", "brute"],
         'leaf_size': [30, 90, 270],
         'p': [1, 2]}

    #decision tree
    hp_dt = {'criterion': ["mse", "friedman_mse", "mae"],
        'splitter': ["best", "random"],
        'max_depth': list(range(2, 51)) + [None],
        'min_samples_split': list(range(2, 51)),
        'min_samples_leaf': list(range(1, 51)),
        'max_features': ["auto", "sqrt", "log2"]}
    
    #support vector machine
    hp_svr = {'kernel': ["linear", "poly", "rbf", "sigmoid"],
         'gamma': ["scale", "auto"],
         'C': list(range(1, 51)),
         'max_iter': [1000, 3000, 9000]}

    #multilayer perceptron or feed-forward neural network
    hp_mlp = {'hidden_layer_sizes': [(np.random.randint(1, 900),), (np.random.randint(1, 900), np.random.randint(1, 900)), 
                                     (np.random.randint(1, 900), np.random.randint(1, 900), np.random.randint(1, 900))],
         'activation': ["identity", "logistic", "tanh", "relu"],
         'solver': ["lbfgs", "sgd", "adam"],
         'learning_rate': ["constant", "invscaling", "adaptive"],
         'learning_rate_init': [0.01, 0.001, 0.0001],
         'max_iter': [1000, 3000, 9000],
         'warm_start': [False, True],
         'momentum': np.random.uniform(low=0.0, high=1.0, size=50),
         'nesterovs_momentum': [False, True],
         'early_stopping': [False, True],
         'n_iter_no_change': [10, 30, 90],
         'max_fun': [15000, 45000, 135000]}

    #random forest
    hp_random_forest = {'n_estimators': [100, 300, 900],
                   'criterion': ["mse", "mae"],
                   'min_samples_split': list(range(2, 51)),
                   'min_samples_leaf': list(range(1, 51)),
                   'max_features': ["auto", "sqrt", "log2"],
                   'oob_score': [False, True],
                   'warm_start': [False, True]}                   

    #ada boosting
    hp_ada_boost = {'n_estimators': [50, 150, 450],
               'learning_rate': [1, 0.1, 0.01],
               'loss': ["linear", "square", "exponential"]}

    #gradient boosting
    hp_gradient_boost = {'loss': ["ls", "lad", "huber", "quantile"],
                     'learning_rate': [0.1, 0.01, 0.001],
                     'n_estimators': [100, 300, 900],
                     'subsample': [0.1, 0.5, 1.0],
                     'criterion': ["friedman_mse", "mse", "mae"],
                     'min_samples_split': list(range(2, 51)),
                     'min_samples_leaf': list(range(1, 51)),
                     'max_depth': list(range(3, 51)),
                     'max_features': ["auto", "sqrt", "log2"],
                     'warm_start': [False, True], 
                     'n_iter_no_change': [10, 30, 90, None]}

    #simple rule regressor
    hp_dummy = {'strategy': ["mean", "median", "quantile"], 
            'quantile': [0.0, 0.25, 0.75, 1.0]}
    
    if verbose:
        print("###### get_estimators_hyperparameters_to_search() ######")
        print(hp_lr)
        print(hp_sgd)
        print(hp_lasso)
        print(hp_bayesian)
        print(hp_knn)
        print(hp_dt)
        print(hp_svr)
        print(hp_mlp)
        print(hp_random_forest)
        print(hp_ada_boost)
        print(hp_gradient_boost)
        print(hp_dummy, "\n")
        
    return hp_lr, hp_sgd, hp_lasso, hp_bayesian, hp_knn, hp_dt, hp_svr, hp_mlp, hp_random_forest, hp_ada_boost, hp_gradient_boost, hp_dummy

In [105]:
def save_estimator(estimator, file_name : str, verbose=True):
    dump(estimator, str(args.model_path + file_name + '.joblib')) 
    
    if verbose:
        print("###### save_estimator() ######")
        print("estimator:", estimator)
        print("saved file:", str(args.model_path + file_name + '.joblib'), "\n")        

In [106]:
def train_and_test_whole(estimator, X_train, y_train, X_test, y_test, verbose=True):
    tic = time.time()
    estimator.fit(X_train, y_train)
    toc = time.time()    
    if verbose: print("Training DONE")
    
    y_pred = estimator.predict(X_test)
    if verbose: print("Testing DONE\n\n")
    
    if verbose:
        print("###### train_and_test_whole() ######")
        print("estimator:", estimator)
        print("X_train:", X_train[:2])
        print("y_train:", y_train[:2])
        print("X_test:", X_test[:2])
        print("y_test:", y_test[:2], "\n")
        
    return y_pred, (toc - tic)

In [107]:
def save_results(performance_dict, data, subset, n, name, verbose=True): 
    df_results = pd.DataFrame.from_dict(performance_dict)
    #i.e 'nasbench201_predictions_cifar10valid_200epochs_n172.csv'
    # 'nasbench201_predictions_cifar100_200epochs_n172.csv'
    # 'nasbench201_predictions_imagenet_16_120_200epochs_n172.csv'
    df_results.to_csv(str(args.results_path + 'nasbench201_' + name + '_' 
                          + str(data) + '_' + str(subset) + 'epochs_n' 
                          + str(n) + '.csv'), index=False, float_format='%.6f')
    
    if verbose:
        print("###### save_results() ######")
        print("saved file:", str(args.results_path + 'nasbench201_' + name + '_' 
                                 + str(data) + '_' + str(subset) + 'epochs_n' + 
                                 str(n) + '.csv'), "\n")   

In [108]:
def train(verbose=True):
    for data in args.dataset:        
        
        for subset in args.data_subset:
        
            #cifar10valid (4, 108, 200), cifar100 and imagenet16_120 (200)
            if data in ('cifar100', 'imagenet16_120') and subset in (4, 108):
                continue #above code is not executed
                             
            X, y = get_preprocessed_numpy_dataset(data, subset)

            for n in args.train_size:
                predictions = {'dataset': [], 'epoch': [], 'train_size': [], 'seed': [], 'model': [], 
                                   'id_arch': [], 'acc_valid_pred': [], 'acc_valid_true': [], 'acc_test_true': []}
                fit_times = {'dataset': [], 'epoch': [], 'train_size': [], 'seed': [], 'model': [], 'fit_time': []}

                for seed in args.seed:
                    if verbose: 
                        print("\n\n\n=========================================================")
                        print("=========== {}, Subset{}, N{}, Seed{} ===========".format(data, subset, n, seed))
                        print("=========================================================")
                    
                    set_default_seed(seed)      
                    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n, random_state=seed, shuffle=True)                    
                    if verbose:
                        print("###### hold-out process ######")
                        print("len(X_train):", len(X_train))
                        print("len(X_test):", len(X_test))
                        print("len(y_train):", len(y_train))
                        print("len(y_test):", len(y_test))

                    min_max_scaler = MinMaxScaler()
                    min_max_scaler.fit(X_train[:, 1:-1]) #ignore id_arch and acc_test columns
                    X_train[:, 1:-1] = min_max_scaler.transform(X_train[:, 1:-1])
                    X_test[:, 1:-1] = min_max_scaler.transform(X_test[:, 1:-1])                    
                    
                    if verbose:
                        print("\n###### Normalization process ######")
                        print("X_train:", X_train[:2])
                        print("X_test:", X_test[:2])

                    estimators = get_estimators(seed)
                    hyperparameters = get_estimators_hyperparameters_to_search()

                    for reg, hp in zip(estimators, hyperparameters):
                        rs = RandomizedSearchCV(estimator=reg, param_distributions=hp, n_iter=args.n_iter_rs, scoring=args.scoring_rs, 
                                                n_jobs=-1, cv=args.cv_inner, verbose=0, pre_dispatch='2*n_jobs', random_state=seed)

                        reg_name = str(reg)[:str(reg).index('(')]
                        if verbose: print("\n#", reg_name, "#")

                        if verbose: print("\n#########", args.cv_inner, "Fold RANDOM SEARCH #########")
                        #id_arch and acc_test columns are not included
                        rs.fit(X_train[:, 1:-1], y_train) 
                        if verbose: print("DONE")

                        best_estimator = rs.best_estimator_
                        if verbose: print("best estimator:", best_estimator.get_params())

                        if verbose: print("\n######### HOLD-OUT VALIDATION #########")
                        y_pred, fit_time = train_and_test_whole(best_estimator, X_train[:, 1:-1], y_train, X_test[:, 1:-1], y_test)                       

                        predictions['dataset'].extend([data] * len(y_pred))
                        predictions['epoch'].extend([subset] * len(y_pred))
                        predictions['train_size'].extend([n] * len(y_pred))
                        predictions['seed'].extend([seed] * len(y_pred))
                        predictions['model'].extend([reg_name] * len(y_pred))
                        predictions['id_arch'].extend(X_test[:, 0]) #id_arch column                        
                        predictions['acc_valid_pred'].extend(y_pred)
                        predictions['acc_valid_true'].extend(y_test)
                        predictions['acc_test_true'].extend(X_test[:, -1]) #acc_test column 

                        fit_times['dataset'].append(data)
                        fit_times['epoch'].append(subset)
                        fit_times['train_size'].append(n)
                        fit_times['seed'].append(seed)
                        fit_times['model'].append(reg_name)
                        fit_times['fit_time'].append(fit_time)

                        save_estimator(best_estimator, str('nasbench201_' + str(data) + '_' + str(subset) + 'epochs_n' 
                                                               + str(n) + '_seed' + str(seed) + '_' + reg_name))

                #predictions and times for ALL models and ALL seeds, per each 1 dataset, 1 subset (epoch), and 1 n
                save_results(predictions, data, subset, n, "predictions")
                save_results(fit_times, data, subset, n, "fit_times")

In [111]:
if __name__ == '__main__':
    print("data_path:", args.data_path)
    print("model_path:", args.model_path)
    print("results_path:", args.results_path)
    print("target:", args.target)
    print("n_iter_rs:", args.n_iter_rs)
    print("cv_inner:", args.cv_inner)
    print("scoring_rs:", args.scoring_rs)
    print("dataset:", args.dataset)
    print("data_subset:", args.data_subset)
    print("seed:", args.seed)
    print("train_size:", args.train_size)                
    print("estimators:", args.estimators)
    print("features drop:", args.features_drop, "\n\n\n")
    
    train()

data_path: /home/gean/nns_performance_prediction/meta_datasets/
model_path: /home/gean/nns_performance_prediction/saved_models/
results_path: /home/gean/nns_performance_prediction/results/fast/test12/
target: acc_valid
n_iter_rs: 2
cv_inner: 3
scoring_rs: r2
dataset: ['imagenet16_120']
data_subset: [4, 108, 200]
seed: [0, 1, 10, 42, 100, 123, 666, 1000, 1234, 12345]
train_size: [43, 86, 129, 172, 344, 860]
estimators: ['linear_regression', 'sgd', 'lasso', 'bayesian_ridge', 'knn', 'dt', 'svm', 'mlp', 'random_forest', 'ada_boost', 'gradient_boost', 'dummy']
features drop: ['str_arch', 'layers_all', 'adjacency_matrix', 'dataset', 'epoch', 'skip_connection_num_layers', 'zeroize_num_layers', 'flops', 'latency', 'time_train', 'time_valid', 'time_test', 'loss_train', 'loss_valid', 'loss_test', 'acc_train'] 



###### get_preprocessed_numpy_dataset() ######
df_whole after pre-processing:     id_arch  conv_num_layers  conv_kernel_min  conv_kernel_max  \
17        0                2             

/home/gean/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


Testing DONE


###### train_and_test_whole() ######
estimator: RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=46,
                      min_samples_split=20, min_weight_fraction_leaf=0.0,
                      n_estimators=900, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=True)
X_train: [[0.33333333 0.33333333 1.         0.33333333 0.         0.26361656]
 [0.33333333 1.         1.         1.         1.         0.47276688]]
y_train: [42.17777767 35.39999997]
X_test: [[0.33333333 0.33333333 1.         0.33333333 0.33333333 0.26361656]
 [0.66666667 0.33333333 1.         0.33333333 0.33333333 0.52723312]]
y_test: [17.55555553 33.63333324] 

###### save_estimator() ######
estimator: RandomForestRegressor(bootstrap=True

/home/gean/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


Testing DONE


###### train_and_test_whole() ######
estimator: RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=18,
                      min_samples_split=38, min_weight_fraction_leaf=0.0,
                      n_estimators=900, n_jobs=-1, oob_score=False,
                      random_state=1, verbose=0, warm_start=True)
X_train: [[0.25       1.         1.         1.         0.25       0.25      ]
 [0.5        0.33333333 1.         0.33333333 0.25       0.27880184]]
y_train: [24.59999995 39.6555554 ]
X_test: [[0.25       1.         1.         1.         0.25       0.25      ]
 [0.5        0.33333333 0.33333333 0.33333333 0.         0.05760369]]
y_test: [39.22222221 29.75555558] 

###### save_estimator() ######
estimator: RandomForestRegressor(bootstrap=True

...

/home/gean/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


Testing DONE


###### train_and_test_whole() ######
estimator: RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=12,
                      min_samples_split=7, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=-1, oob_score=True,
                      random_state=10, verbose=0, warm_start=True)
X_train: [[0.4        0.33333333 1.         0.33333333 0.5        0.27099664]
 [0.4        0.33333333 0.33333333 0.33333333 0.         0.05599104]]
y_train: [36.86666662 38.02222218]
X_test: [[0.6        1.         1.         1.         0.25       0.72900336]
 [0.4        0.33333333 0.33333333 0.33333333 0.25       0.05599104]]
y_test: [17.79999992 35.68888883] 

###### save_estimator() ######
estimator: RandomForestRegressor(bootstrap=True,

/home/gean/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


DONE
best estimator: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (103,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0001, 'max_fun': 135000, 'max_iter': 3000, 'momentum': 0.9695846277645586, 'n_iter_no_change': 30, 'nesterovs_momentum': False, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': True}

######### HOLD-OUT VALIDATION #########
Training DONE
Testing DONE


###### train_and_test_whole() ######
estimator: MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(103,), learning_rate='adaptive',
             learning_rate_init=0.0001, max_fun=135000, max_iter=3000,
             momentum=0.9695846277645586, n_iter_no_change=30,
             nesterov

KeyboardInterrupt: 

In [ ]:
!which python

In [ ]:
!python --version

In [ ]:
!pip list